# Calibrate cameras, create initial orthoimage and partial DSM

In [1]:
import os
from glob import glob
import subprocess
import numpy as np
from tqdm import tqdm
import shutil
import rasterio as rio
from affine import Affine
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr
from shapely.geometry import Point
import geopandas as gpd
# Ignore warnings (rasterio throws a warning whenever an image is not georeferenced. Annoying in this case.)
import warnings
warnings.filterwarnings('ignore')

# Define input image files
data_folder = '/Users/rdcrlrka/Research/Soo_locks'
img_folder = os.path.join(data_folder, '20251001_imagery', 'frames_IR')
img_list = sorted(glob(os.path.join(img_folder, '*.tiff')))
print(f"{len(img_list)} images located")

# Grab standard input files
refdem_file = os.path.join(os.getcwd(), 'inputs', '20251001_Soo_Model_1cm_Intensity_UTM19N-fake.tif')
gcp_folder = os.path.join(os.getcwd(), 'inputs', 'gcp')
cams_file = os.path.join(os.getcwd(), 'inputs', 'cams_lonlat-fake.txt')

# Define output folders
out_folder = img_folder + '_proc_out'
os.makedirs(out_folder, exist_ok=True)
new_img_folder = os.path.join(out_folder, 'single_band_images')
undistorted_folder = os.path.join(out_folder, 'undistorted_images')
cam_folder = os.path.join(out_folder, 'cam_gen')

ba_folder = os.path.join(out_folder, 'bundle_adjust')
init_ortho_folder = os.path.join(out_folder, 'init_ortho')

16 images located


## Convert images to single band in case they're RGB

A couple IR images (near the windows) were captured in RGB 

In [2]:
os.makedirs(new_img_folder, exist_ok=True)

# iterate over images
print('Saving single-band images to:', new_img_folder)
for img_fn in tqdm(img_list[0:8]):
    # convert images to single band
    out_fn = os.path.join(new_img_folder, os.path.basename(img_fn))
    if os.path.exists(out_fn):
        continue
    cmd = [
        "gdal_translate",
        "-b", "1",
        img_fn, out_fn
    ]
    subprocess.run(cmd)

Saving single-band images to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/single_band_images


100%|██████████| 8/8 [00:00<00:00, 15932.78it/s]


## Undistort images using GCP

In [3]:
# def warp_image_using_gcp(image_undistorted, gcp_df, K, dist, dst_crs="EPSG:32619"):

#     # --- Undistort GCP pixel coordinates ---
#     img_pts = gcp_df[['sample_col', 'sample_row']].values.astype(np.float32).reshape(-1, 1, 2)
#     undistorted_pts = cv2.undistortPoints(img_pts, K, dist, P=K).reshape(-1, 2)
#     gcp_df['sample_col_undist'] = undistorted_pts[:, 0]
#     gcp_df['sample_row_undist'] = undistorted_pts[:, 1]

#     # --- Estimate affine transform from undistorted image pixels to world coords ---
#     img_pts = gcp_df[['sample_col_undist', 'sample_row_undist']].values.astype(np.float32)
#     world_pts = gcp_df[['X', 'Y']].values.astype(np.float32)
#     M, _ = cv2.estimateAffinePartial2D(img_pts, world_pts, method=cv2.LMEDS)

#     # --- Estimate pixel size (resolution) from affine transform ---
#     scale_x = np.linalg.norm(M[:, 0])
#     scale_y = np.linalg.norm(M[:, 1])

#     pixel_size_x = scale_x
#     pixel_size_y = scale_y

#     # --- Compose scaled affine matrix to preserve resolution ---
#     scaling = np.array([
#         [1 / pixel_size_x, 0, 0],
#         [0, 1 / pixel_size_y, 0],
#         [0, 0, 1]
#     ])
#     M_hom = np.vstack([M, [0, 0, 1]])  # 3x3
#     M_scaled = scaling @ M_hom  # Affine matrix with resolution normalization

#     # --- Transform image corners using scaled matrix ---
#     h, w = image_undistorted.shape[:2]
#     corners = np.array([
#         [0, 0],
#         [w, 0],
#         [0, h],
#         [w, h]
#     ], dtype=np.float32)
#     corners_hom = np.hstack([corners, np.ones((4, 1))])
#     transformed_corners = (M_scaled @ corners_hom.T).T

#     # --- Compute bounds in pixel coordinates ---
#     x_coords = transformed_corners[:, 0]
#     y_coords = transformed_corners[:, 1]
#     x_min_px, x_max_px = np.floor(np.min(x_coords)), np.ceil(np.max(x_coords))
#     y_min_px, y_max_px = np.floor(np.min(y_coords)), np.ceil(np.max(y_coords))

#     width = int(x_max_px - x_min_px)
#     height = int(y_max_px - y_min_px)

#     # --- Apply final translation to ensure top-left is (0, 0) in output image ---
#     translation = np.array([
#         [1, 0, -x_min_px],
#         [0, 1, -y_min_px],
#         [0, 0, 1]
#     ])
#     M_final = translation @ M_scaled
#     M_final = M_final[:2, :]  # Back to 2x3 for OpenCV

#     # --- Warp image using final matrix ---
#     image_undistorted_warped = cv2.warpAffine(
#         image_undistorted,
#         M_final,
#         (width, height),
#         flags=cv2.INTER_LINEAR,
#         borderValue=0
#     )
#     # set no data to nan
#     image_undistorted_warped = np.where(image_undistorted_warped==0, np.nan, image_undistorted_warped)

#     # plt.imshow(image_undistorted_warped)
#     # plt.show()

#     # --- Generate geospatial coordinates for warped image ---
#     x_origin = (x_min_px * pixel_size_x)
#     y_origin = (y_min_px * pixel_size_y)

#     x_coords = np.linspace(x_origin, x_origin + pixel_size_x * width, num=width, endpoint=False)
#     y_coords = np.linspace(y_origin, y_origin + pixel_size_y * height, num=height, endpoint=False)

#     dims = ['y', 'x'] if image_undistorted.ndim == 2 else ['y', 'x', 'band']
#     image_da = xr.DataArray(
#         data=image_undistorted_warped,
#         dims=dims,
#         coords=dict(
#             y=y_coords,
#             x=x_coords
#         )
#     )
#     image_da.rio.write_crs(dst_crs, inplace=True)

#     return image_da, gcp_df


def estimate_shared_intrinsics(image_files, gcp_files, output_folder, fx_mm=2.8, plot_results=True):

    object_points_list = []
    image_points_list = []
    image_size = None

    print(f"Preparing data for {len(image_files)} images...")

    for image_file in image_files:
        img_name = os.path.basename(image_file)

        # Load GCP file for this image
        gcp_file = [x for x in gcp_files if (os.path.splitext(img_name)[0] in x)]
        if not gcp_file:
            print(f"No GCP file found for {img_name}, skipping.")
            continue
        gcp_file = gcp_file[0]

        gcp = pd.read_csv(
            gcp_file,
            sep=', ',
            header=None,
            skiprows=[0],
            engine='python',
            names=['pt_idx', 'Y', 'X', 'Z', 'Y_std', 'X_std', 'Z_std',
                    'img_path', 'sample_col', 'sample_row', 'use_Y', 'use_X', 'use_Z']
        )

        # Reproject to UTM (EPSG:32619)
        gcp['geometry'] = [Point(x, y) for x, y in gcp[['X', 'Y']].values]
        gcp_gdf = gpd.GeoDataFrame(geometry=gcp['geometry'], crs="EPSG:4326")
        gcp_gdf = gcp_gdf.to_crs("EPSG:32619")
        gcp['X'] = gcp_gdf.geometry.x
        gcp['Y'] = gcp_gdf.geometry.y

        if len(gcp) < 6:
            print(f"Skipping {img_name}: only {len(gcp)} GCPs found.")
            continue

        # Prepare object and image points
        object_points = gcp[["X", "Y", "Z"]].values.astype(np.float32).reshape(-1, 1, 3)
        object_points -= object_points.mean(axis=0)  # center around origin

        image_points = gcp[["sample_col", "sample_row"]].values.astype(np.float32)
        image_points = image_points.reshape(-1, 1, 2)

        img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Failed to read {image_file}, skipping.")
            continue

        if image_size is None:
            h, w = img.shape
            image_size = (w, h)
        else:
            assert image_size == (img.shape[1], img.shape[0]), "All images must have the same size"

        object_points_list.append(object_points)
        image_points_list.append(image_points)

    if len(object_points_list) == 0:
        raise ValueError("No valid GCPs found for any images.")

    # --- Calibration ---
    fx = fy = 2000
    cx = image_size[0] / 2
    cy = image_size[1] / 2
    K_init = np.array([[fx, 0, cx],
                       [0, fy, cy],
                       [0,  0,  1]], dtype=np.float64)
    dist_init = np.zeros(5)

    flags = (
        cv2.CALIB_USE_INTRINSIC_GUESS |
        cv2.CALIB_FIX_PRINCIPAL_POINT |
        cv2.CALIB_ZERO_TANGENT_DIST
    )

    print(f"Running calibration on {len(object_points_list)} images...")

    ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
        object_points_list,
        image_points_list,
        image_size,
        K_init,
        dist_init,
        flags=flags
    )

    print("Calibration complete.")
    print("RMS reprojection error:", ret)
    print("Camera Matrix (K):\n", K)
    print("Distortion Coefficients:", dist.ravel())

    os.makedirs(output_folder, exist_ok=True)

    # --- Undistort and georeference each image ---
    for image_file in tqdm(image_files):
        img_name = os.path.basename(image_file)

        # Load the GCPs
        gcp_file = [x for x in gcp_files if (os.path.splitext(img_name)[0] in x)]
        if not gcp_file:
            continue
        gcp_file = gcp_file[0]
        gcp = pd.read_csv(
            gcp_file,
            sep=', ',
            header=None,
            skiprows=[0],
            engine='python',
            names=['pt_idx', 'Y', 'X', 'Z', 'Y_std', 'X_std', 'Z_std',
                   'img_path', 'sample_col', 'sample_row', 'use_Y', 'use_X', 'use_Z']
        )

        # Reproject GCPs again
        gcp['geometry'] = [Point(x, y) for x, y in gcp[['X', 'Y']].values]
        gcp_gdf = gpd.GeoDataFrame(geometry=gcp['geometry'], crs="EPSG:4326")
        gcp_gdf = gcp_gdf.to_crs("EPSG:32619")
        gcp['X'] = gcp_gdf.geometry.x
        gcp['Y'] = gcp_gdf.geometry.y

        # Read the raw image
        img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        # Undistort the image and the GCP
        map1, map2 = cv2.initUndistortRectifyMap(K, dist, None, K, image_size, cv2.CV_32FC1)
        img_undistorted = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR)

        # Calculate new GCP pixel indices for the undistorted image
        img_pts = gcp[['sample_col', 'sample_row']].values.astype(np.float32).reshape(-1, 1, 2)
        undistorted_pts = cv2.undistortPoints(img_pts, K, dist, P=K).reshape(-1, 2)
        gcp['sample_col_undist'] = undistorted_pts[:, 0]
        gcp['sample_row_undist'] = undistorted_pts[:, 1]
        
        if plot_results:
            fig, ax = plt.subplots(1, 2, figsize=(14, 5))
            ax[0].imshow(img, cmap='gray')
            ax[0].plot(gcp['sample_col'], gcp['sample_row'], 'xr',
                       markersize=5, linewidth=1.5)
            ax[0].set_title('Original')
            ax[1].imshow(img_undistorted, cmap='gray')
            ax[1].plot(gcp['sample_col_undist'], gcp['sample_row_undist'], 'xr',
                       markersize=5, linewidth=1.5)
            ax[1].set_title('Undistorted')
            for axis in ax:
                axis.set_xticks([]), axis.set_yticks([])
            plt.suptitle(img_name)
            plt.tight_layout()

            # save to file
            fig_fn = os.path.join(output_folder, os.path.splitext(img_name)[0] + '_undistorted.png')
            fig.savefig(fig_fn, dpi=300, bbox_inches='tight')

            plt.close()

        # Save undistorted image as GeoTIFF
        img_undistorted_file = os.path.join(output_folder, os.path.splitext(img_name)[0] + '_undistorted.tiff')
        img_undistorted_xr = xr.DataArray(
            data=np.flipud(img_undistorted),
            dims=['y', 'x'],
            coords=dict(
                y=np.arange(0,img_undistorted.shape[0]),
                x=np.arange(0,img_undistorted.shape[1])
            )
        )
        img_undistorted_xr.rio.to_raster(img_undistorted_file)
        print('Saved undistorted raster to:', img_undistorted_file)

        # Reformat GCP for saving
        # reproject to lat-lon (fake)
        gcp['geometry'] = [Point(x,y) for x,y in gcp[['X','Y']].values]
        gcp_gdf = gpd.GeoDataFrame(geometry=gcp['geometry'], crs="EPSG:32619")
        gcp_gdf = gcp_gdf.to_crs("EPSG:4326")
        gcp['X'] = [x.coords.xy[0][0] for x in gcp_gdf['geometry']]
        gcp['Y'] = [x.coords.xy[1][0] for x in gcp_gdf['geometry']]
        # update the image name
        gcp['img_name'] = [os.path.basename(x).replace('.tiff', '_undistorted.tiff') for x in gcp['img_path']]
        # select the appropriate rows in order
        gcp['sample_col'] = gcp['sample_col_undist']
        gcp['sample_row'] = gcp['sample_row_undist']
        gcp = gcp[['pt_idx', 'Y', 'X', 'Z', 'Y_std', 'X_std', 'Z_std', 'img_name', 'sample_col', 'sample_row', 'use_Y', 'use_X']]

        # Save undistorted GCP as CSV        
        gcp_undistorted_file = os.path.join(output_folder, os.path.splitext(os.path.basename(gcp_file))[0] + '_undistorted.gcp')
        gcp.to_csv(
            gcp_undistorted_file, 
            sep=',', 
            index=False,
            header=False
            )
        print('Saved undistorted GCP to:', gcp_undistorted_file)

    return img_undistorted, gcp


image_list = sorted(glob(os.path.join(new_img_folder, '*.tiff')))
gcp_list = sorted(glob(os.path.join(gcp_folder, '*.gcp')))

# Process first group of images
image_list_group1 = image_list[0:6]
img_undistorted, gcp = estimate_shared_intrinsics(
    image_list_group1,
    gcp_list,
    undistorted_folder,
    plot_results=True
)


Preparing data for 6 images...
Running calibration on 6 images...
Calibration complete.
RMS reprojection error: 24.599006189509957
Camera Matrix (K):
 [[4.44460121e+03 0.00000000e+00 1.92000000e+03]
 [0.00000000e+00 4.46098142e+03 1.08000000e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients: [-1.45377416  3.50293614  0.          0.         -4.26900282]


 17%|█▋        | 1/6 [00:01<00:09,  1.80s/it]

Saved undistorted raster to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch01_main_20251001180000_20251001180626_undistorted.tiff
Saved undistorted GCP to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch01_main_20251001180000_20251001180626_undistorted.gcp


 33%|███▎      | 2/6 [00:03<00:07,  1.76s/it]

Saved undistorted raster to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch02_main_20251001180001_20251001180626_undistorted.tiff
Saved undistorted GCP to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch02_main_20251001180001_20251001180626_undistorted.gcp


...

Saved undistorted raster to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch04_main_20251001180003_20251001180626_undistorted.tiff
Saved undistorted GCP to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch04_main_20251001180003_20251001180626_undistorted.gcp


 83%|████████▎ | 5/6 [00:08<00:01,  1.74s/it]

Saved undistorted raster to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch05_main_20251001180004_20251001180626_undistorted.tiff
Saved undistorted GCP to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch05_main_20251001180004_20251001180626_undistorted.gcp


100%|██████████| 6/6 [00:10<00:00,  1.74s/it]

Saved undistorted raster to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch06_main_20251001180005_20251001180626_undistorted.tiff
Saved undistorted GCP to: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/undistorted_images/N910A6_ch06_main_20251001180005_20251001180626_undistorted.gcp


## Generate initial camera models

In [4]:
def generate_cameras(image_list, gcp_list, cams_file, f_m=2.8*1e-3, gsd_m=2.5*1e-3, altitude_m=4.3):
    # Load "lat-lon" camera positions
    cams = pd.read_csv(
        cams_file,
        sep=' ',
        header=None,
        names=['img_name', 'lon', 'lat', 'Z', 'lon_std', 'lat_std']
        )

    # Estimate initial camera intrinsics in pixels
    # convert focal length from mm to pixels
    # GSD = (Altidude * Pixel Pitch) / Focal Length
    # --> Pixel Pitch = (GSD * Focal Length) / Altitude
    px_pitch = (gsd_m * f_m) / altitude_m
    # Convert focal length to pixels to use a pitch of 1
    f_px = f_m / px_pitch
    print('Focal length (pixels):', f_px)

    pbar = tqdm(total=len(image_list))
    for image_file in image_list:
        image_file_base = os.path.splitext(os.path.basename(image_file))[0].replace('_undistorted','')

        # Subset camera positions
        cam = cams.loc[cams['img_name'].str.contains(image_file_base)]

        # Load GCP
        gcp_file = [x for x in gcp_list if image_file_base in x]
        if len(gcp_file) < 1:
            raise ValueError(f"No GCP file found for image: {image_file}")
        gcp = pd.read_csv(
            gcp_file[0], 
            sep=',', 
            header=None,
            names=['pt_idx', 'Y', 'X', 'Z', 'Y_std', 'X_std', 'Z_std', 'img_name', 'sample_col', 'sample_row', 'use_Y', 'use_X']
            )

        # get "lat-lon" values and image pixel indices pairs
        lonlat_str = [(str(lon),str(lat)) for lon,lat in gcp[['X', 'Y']].values]
        lonlat_str = [' '.join(xy) for xy in lonlat_str]
        lonlat_str = ', '.join(lonlat_str)

        pxval_str = [(str(c),str(r)) for c,r in gcp[['sample_col', 'sample_row']].values]
        pxval_str = [' '.join(xy) for xy in pxval_str]
        pxval_str = ', '.join(pxval_str)

        # Image dimensions
        with rio.open(image_file) as src:
            w_px = src.width
            h_px = src.height

        # Optical center
        cu = w_px / 2
        cv = h_px / 2

        # construct command
        cmd = [
            'cam_gen',
            # input image file
            image_file,
            '--threads', '12',
            '--camera-type', 'pinhole',
            '--refine-camera',
            '--reference-dem', refdem_file,
            # default height where DEM is NaN
            '--height-above-datum', str(float(np.round(gcp['Z'].mean()))),
            '--focal-length', str(f_px),
            '--pixel-pitch', '1',
            # horizontal, vertical components
            '--optical-center', str(int(cu)), str(int(cv)),
            # lon, lat, height
            '--camera-center-llh', str(cam['lon'].values[0]), str(cam['lat'].values[0]), str(cam['Z'].values[0]),
            '--lon-lat-values', lonlat_str,
            '--pixel-values', pxval_str,
            # output camera
            '-o', os.path.join(
                cam_folder, 
                os.path.splitext(os.path.basename(image_file))[0] + '.tsai'
                ),
        ]
        subprocess.run(cmd)
        pbar.update(1)

os.makedirs(cam_folder, exist_ok=True)

img_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tiff')))[0:6]
gcp_list = sorted(glob(os.path.join(undistorted_folder, '*.gcp')))

generate_cameras(img_list, gcp_list, cams_file)

Focal length (pixels): 1720.0


  0%|          | 0/6 [00:00<?, ?it/s]

	--> Setting number of processing threads to: 12
Using datum: Geodetic Datum --> Name: WGS_1984  Spheroid: WGS 84  Semi-major axis: 6378137  Semi-minor axis: 6356752.3142451793  Meridian: Greenwich at 0  Proj4 Str: +proj=longlat +datum=WGS84 +no_defs
Using nodata value: -9999
Camera center (lon-lat-height) set on the command line: Vector3(-73.488768183874271,3.4031005725631767e-05,-3.863)
Could not determine a valid height value at lon-lat: -73.488759911372853 4.4212098441672778e-05. Will use a height of -8.
Median pixel projection error in the coarse camera: 1761.7551854342082


 17%|█▋        | 1/6 [00:02<00:10,  2.06s/it]

The Levenberg-Marquardt solver failed. Results may be inaccurate.
Median pixel projection error in the refined camera: 2099.7879078997926
Output camera center lon, lat, and height above datum: Vector3(-73.48875751848125,4.4030130947677164e-05,-7.4758407596162746)
Writing: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/cam_gen/N910A6_ch01_main_20251001180000_20251001180626_undistorted.tsai


...

	--> Setting number of processing threads to: 12
Using datum: Geodetic Datum --> Name: WGS_1984  Spheroid: WGS 84  Semi-major axis: 6378137  Semi-minor axis: 6356752.3142451793  Meridian: Greenwich at 0  Proj4 Str: +proj=longlat +datum=WGS84 +no_defs
Using nodata value: -9999
Camera center (lon-lat-height) set on the command line: Vector3(-73.488795429984549,0.00013154758682599999,-3.6419999999999999)
Median pixel projection error in the coarse camera: 1664.0953326868441
Median pixel projection error in the refined camera: 1336.7930639518172
Output camera center lon, lat, and height above datum: Vector3(-73.489980407895032,-0.00071540038317027496,47099.745333227052)
Writing: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/cam_gen/N910A6_ch04_main_20251001180003_20251001180626_undistorted.tsai


 83%|████████▎ | 5/6 [00:08<00:01,  1.71s/it]

	--> Setting number of processing threads to: 12
Using datum: Geodetic Datum --> Name: WGS_1984  Spheroid: WGS 84  Semi-major axis: 6378137  Semi-minor axis: 6356752.3142451793  Meridian: Greenwich at 0  Proj4 Str: +proj=longlat +datum=WGS84 +no_defs
Using nodata value: -9999
Camera center (lon-lat-height) set on the command line: Vector3(-73.48880679806058,0.0001618328422275,-3.532)
Median pixel projection error in the coarse camera: 1414.7086676363992
Median pixel projection error in the refined camera: 1638.775779336627
Output camera center lon, lat, and height above datum: Vector3(-73.483068419337698,0.0015474428795117473,8665.4174038834444)
Writing: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/cam_gen/N910A6_ch05_main_20251001180004_20251001180626_undistorted.tsai


100%|██████████| 6/6 [00:10<00:00,  1.74s/it]

	--> Setting number of processing threads to: 12
Using datum: Geodetic Datum --> Name: WGS_1984  Spheroid: WGS 84  Semi-major axis: 6378137  Semi-minor axis: 6356752.3142451793  Meridian: Greenwich at 0  Proj4 Str: +proj=longlat +datum=WGS84 +no_defs
Using nodata value: -9999
Camera center (lon-lat-height) set on the command line: Vector3(-73.488813745770074,0.0001955598913277,-3.5169999999999999)
Median pixel projection error in the coarse camera: 2862.1605171374395
Median pixel projection error in the refined camera: 1477.0094161453283
Output camera center lon, lat, and height above datum: Vector3(-73.495468703809294,-0.0053102586722478945,12610.625966402533)
Writing: /Users/rdcrlrka/Research/Soo_locks/20251001_imagery/frames_IR_proc_out/cam_gen/N910A6_ch06_main_20251001180005_20251001180626_undistorted.tsai


## Align images to the lidar reflectance image using GCP

In [ ]:
os.makedirs(ba_folder, exist_ok=True)

img_list = sorted(glob(os.path.join(undistorted_folder, '*.tiff')))
gcp_list = sorted(glob(os.path.join(undistorted_folder, '*.gcp')))
cam_list = sorted(glob(os.path.join(cam_folder, '*.tsai')))

# run bundle_adjust in groups of two images for better optimization
for i in np.arange(0, len(img_list)-1, step=2):
    img1, img2 = img_list[i:i+2]
    cam1, cam2 = cam_list[i:i+2]
    gcp1, gcp2 = gcp_list[i:i+2]
    
    pair_prefix = os.path.join(
        ba_folder, 
        '__'.join([os.path.splitext(os.path.basename(x))[0] for x in (img1, img2)]),
        'run'
        )
    
    cmd = [
    'parallel_bundle_adjust',
        '-t', 'pinhole',
        '--threads', '12',
        '--num-iterations', '2000',
        '--num-passes', '2',
        # create new camera files
        '--inline-adjustments',
        '--heights-from-dem', refdem_file,
        '--heights-from-dem-uncertainty', '0.02',
        '--remove-outliers-params', "75.0 3.0 20 25",
        '-o', pair_prefix,
        img1, img2,
        cam1, cam2,
        gcp1, gcp2
    ]
    subprocess.run(cmd)


## Mapproject

In [ ]:
os.makedirs(init_ortho_folder, exist_ok=True)

img_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tiff')))
cam_list = sorted(glob(os.path.join(ba_folder, '*', '*.tsai')))

pbar = tqdm(total=len(img_list))
for img_file, cam_file in zip(img_list, cam_list):
    img_out_file = os.path.join(init_ortho_folder, os.path.basename(img_file).replace('.tiff', '_map.tiff'))
    cmd = [
        'mapproject',
        '--threads', '12',
        '--tr', '0.005',
        refdem_file, img_file, cam_file, img_out_file
    ]
    subprocess.run(cmd)
    pbar.update(1)

In [ ]:
# # align photos using the undistorted GCP
# gcp_list = sorted(glob(os.path.join(undistorted_folder, '*.gcp')))
# img_list = sorted(glob(os.path.join(undistorted_folder, '*.tiff')))

# def compute_georeferencing_transform(gcp_df):
#     """
#     Compute a similarity transform (rotation, scale, translation) from image pixels to UTM coords.
#     Returns an Affine transform object usable with rasterio.
#     """

#     # Use undistorted GCP pixel coordinates
#     img_pts = gcp_df[['sample_col_undist', 'sample_row_undist']].values.astype(np.float32)
#     world_pts = gcp_df[['X', 'Y']].values.astype(np.float32)

#     # Estimate similarity transform
#     M, inliers = cv2.estimateAffinePartial2D(img_pts, world_pts, method=cv2.LMEDS)

#     if M is None:
#         raise RuntimeError("Could not estimate transform.")

#     # Convert OpenCV affine matrix (2x3) to rasterio-style Affine
#     a, b, c = M[0]
#     d, e, f = M[1]

#     transform = Affine(a, b, c, d, e, f)
#     print(transform)
    
#     return transform


# os.makedirs(align_cv_folder, exist_ok=True)

# for gcp_file in tqdm(gcp_list[0:1]):
#     # open the GCP file
#     gcp = pd.read_csv(
#             gcp_file,
#             sep=' ',
#             header=None,
#             skiprows=[0],
#             engine='python',
#             names=['Y', 'X', 'Z', 'Y_std', 'X_std', 'Z_std',
#                     'img_path', 'sample_col', 'sample_row', 'use_Y', 'use_X', 'use_Z', 'geometry',
#                     'sample_col_undist', 'sample_row_undist']
#         )
    
#     # get the image file name
#     img_name_base = os.path.splitext(os.path.basename(gcp.iloc[0]['img_path']))[0]
#     img_file = [x for x in img_list if img_name_base in x][0]

#     # load the original transform
#     img = rxr.open_rasterio(img_file).squeeze()
#     img_transform = img.rio.transform()

#     # estimate the alignment transform
#     align_transform = compute_georeferencing_transform(gcp)

#     # apply alignment transform to the original
#     img_transform_aligned = align_transform * img_transform

#     # apply the transform and CRS to a new DataArray
#     img_aligned = img.copy()
#     img_aligned.rio.write_transform(img_transform_aligned, inplace=True)
#     img_aligned.rio.write_crs(img.rio.crs, inplace=True)

#     # Save output
#     img_out_fn = os.path.join(align_cv_folder, os.path.basename(img_file).replace('.tiff', '_aligned.tiff'))
#     img_aligned.rio.to_raster(img_out_fn)

#     print('Aligned image saved to:', img_out_fn)
    

In [ ]:
# gcp = pd.read_csv(
#     gcp_list[3],
#     sep=' ',
#     header=None,
#     skiprows=[0],
#     engine='python',
#     names=['pt_idx', 'Y', 'X', 'Z', 'Y_std', 'X_std', 'Z_std',
#             'img_path', 'sample_col', 'sample_row', 'use_Y', 'use_X', 'use_Z', 'geometry', 'sample_col_undist', 'sample_row_undist']
# )

# # Reproject GCPs again
# gcp['geometry'] = [Point(x, y) for x, y in gcp[['X', 'Y']].values]
# gcp_gdf = gpd.GeoDataFrame(geometry=gcp['geometry'], crs="EPSG:4326")
# gcp_gdf = gcp_gdf.to_crs("EPSG:32619")
# gcp['X'] = gcp_gdf.geometry.x
# gcp['Y'] = gcp_gdf.geometry.y
# gcp['image_name'] = [os.path.basename(x) for x in gcp['img_path']]
# gcp.rename(columns={
#     'sample_col_undist': 'sample_col_undistorted',
#     'sample_row_undist': 'sample_row_undistorted'
# }, inplace=True)

# gcp = gcp[['X', 'Y', 'Z', 'image_name', 'sample_col', 'sample_row', 'sample_col_undistorted', 'sample_row_undistorted']]
# gcp_out = gcp_list[3].replace('.gcp', '.csv')
# gcp.to_csv(gcp_out, sep=',', header=True, index=False)

In [ ]:

# open orthoimage for plotting
ortho_file = os.path.join(os.getcwd(), 'inputs', '20251001_Soo_Model_5mm_Ground_Reflectance_UTM19N-fake.tif')
ortho = rxr.open_rasterio(ortho_file).squeeze()
ortho = ortho.data.astype(np.float32) # openCV only accepts up to 32-bit
ortho[ortho==-9999] = 0

# iterate over match files
for match_txt in match_txt_list[0:1]:
    match_txt_base = os.path.splitext(os.path.basename(match_txt).replace('run-',''))[0]

    # parse the image and ortho pixel coordinates
    match = pd.read_csv(match_txt, sep=' ', header=None, skiprows=[0])
    image_match_pts = match.iloc[0:int(len(match)/2)][[0,1]].values
    ortho_match_pts = match.iloc[int(len(match)/2):][[0,1]].values

    # open the image
    image_file = os.path.join(undistorted_folder, match_txt_base.split('__')[0] + '.tiff')
    image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

    # align
    M, inliers = cv2.estimateAffine2D(np.array(image_match_pts), np.array(ortho_match_pts))
    


# Plot GCP / match points


# fig, ax = plt.subplots(1, 2, figsize=(10,5))
# ax[0].imshow(image, cmap='Grays_r')
# ax[1].imshow(ortho, cmap='Grays_r')
# for i in range(len(image_match)):
#     ax[0].plot(image_match.iloc[i][0], image_match.iloc[i][1], '*', markersize=8,
#                markerfacecolor=plt.cm.viridis(i/len(image_match)), markeredgecolor='w', markeredgewidth=0.3)
#     ax[1].plot(ortho_match.iloc[i][0], ortho_match.iloc[i][1], '*', markersize=8,
#                markerfacecolor=plt.cm.viridis(i/len(image_match)), markeredgecolor='w', markeredgewidth=0.3)
# ax[1].set_xlim(ortho_match[0].min() - 15, ortho_match[0].max() + 15)
# ax[1].set_ylim(ortho_match[1].max() + 15, ortho_match[1].min() - 15)
# for axis in ax:
#     axis.set_xticks([])
#     axis.set_yticks([])

# plt.show()

In [ ]:
h, w = image.shape[:2]

# Step 1: Transform source image corners
corners = np.array([
    [0, 0],
    [w, 0],
    [0, h],
    [w, h]
], dtype=np.float32)

# Convert corners to homogeneous coords
ones = np.ones((4, 1), dtype=np.float32)
corners_hom = np.hstack([corners, ones])

# Apply M to each corner
transformed_corners = (M @ corners_hom.T).T

# Step 2: Compute bounding box of warped image
x_min, y_min = np.min(transformed_corners, axis=0)
x_max, y_max = np.max(transformed_corners, axis=0)

width = int(np.ceil(x_max - x_min))
height = int(np.ceil(y_max - y_min))

# Step 3: Adjust M to shift output image to (0, 0)
translation = np.array([
    [1, 0, -x_min],
    [0, 1, -y_min]
])

M_adjusted = translation @ np.vstack([M, [0, 0, 1]])  # make M 3x3 for matrix mult
M_adjusted = M_adjusted[:2, :]  # back to 2x3 for warpAffine

# Step 4: Warp image with adjusted matrix
warped = cv2.warpAffine(image, M_adjusted, (width, height), flags=cv2.INTER_LINEAR, borderValue=0)

ortho = rxr.open_rasterio(ortho_file).squeeze()
ortho = ortho.data.astype(np.float32) # openCV only accepts up to 32-bit
ortho[ortho==-9999] = 0

plt.imshow(warped)
plt.show()


In [ ]:
# # Add initial distortion coefficients
# cam_list = sorted(glob(os.path.join(cam_folder, '*.tsai')))

# for cam in tqdm(cam_list):
#     with open(cam, 'r') as f:
#         cam_lines = f.read().split('\n')
#     # remove empty lines
#     cam_lines = [x for x in cam_lines if x!='']

#     # replace the NULL with TSAI
#     for i, line in enumerate(cam_lines):
#         if 'NULL' in line:
#             cam_lines[i] = 'TSAI'

#     # add distortion coefficients
#     cam_lines += [
#         'k1 = -1e-6',
#         'k2 = 1e-6',
#         'p1 = 0',
#         'p2 = 0',
#         'k3 = 1e-6'
#     ]

#     cam_lines_string = '\n'.join(cam_lines) + '\n'
    
#     # write to file
#     with open(cam, 'w') as f:
#         f.write(cam_lines_string)

## Run stereo correlation for dense matches

In [ ]:
os.makedirs(init_stereo_folder, exist_ok=True)

img_list = sorted(glob(os.path.join(undistorted_folder, '*.tiff')))
cam_list = [glob(os.path.join(cam_folder, '*' + os.path.splitext(os.path.basename(x))[0]) + '*.tsai')[0]
            for x in img_list]

img_pairs = list(zip(img_list[0:-1], img_list[1:]))
cam_pairs = list(zip(cam_list[0:-1], cam_list[1:]))

for i in tqdm(range(0,len(img_pairs))):
    img1, img2 = img_pairs[i]
    cam1, cam2 = cam_pairs[i]

    pair_prefix = os.path.join(
        init_stereo_folder,
        os.path.splitext(os.path.basename(img1))[0] + '__' + os.path.splitext(os.path.basename(img2))[0],
        'run'
        )

    cmd = [
        'parallel_stereo',
        '--threads-singleprocess', '12',
        '--threads-multiprocess', '12',
        '--stop-point', '1', # stop after feature detection and matching
        img1, img2,
        cam1, cam2,
        pair_prefix
    ]
    subprocess.run(cmd)

## Bundle adjust using dense matches

In [ ]:
os.makedirs(ba_folder, exist_ok=True)

cam_list = sorted(glob(os.path.join(cam_folder, '*.tsai')))
img_list = [os.path.join(new_img_folder, os.path.basename(x).replace('.tsai','.tiff')) 
            for x in cam_list]

# copy dense matches from stereo to bundle_adjust folder
match_list = glob(os.path.join(init_stereo_folder, '*', '*.match'))
print(f'Copying {len(match_list)} matches from stereo to the bundle adjust folder')
for match in tqdm(match_list):
    match_out = os.path.join(ba_folder, os.path.basename(match))
    _ = shutil.copy2(match, match_out)

cmd = [
    'parallel_bundle_adjust',
    '-t', 'pinhole',
    '--threads', '12',
    '--num-iterations', '2000',
    '--num-passes', '2',
    # create new camera files
    '--inline-adjustments',
    # more confident in the camera positions
    # '--camera-position-weight', '5',
    # use the matches from stereo
    '--force-reuse-match-files',
    # solve intrinsics
    # '--solve-intrinsics',
    # '--intrinsics-to-share', 'none',
    # '--intrinsics-to-float', 'all',
    '--heights-from-dem', refdem_file,
    '--heights-from-dem-uncertainty', '0.02',
    '--remove-outliers-params', "75.0 3.0 20 25",
    '-o', os.path.join(ba_folder, 'run'),
] + img_list + cam_list
subprocess.run(cmd)



## Initial orthorectification

In [ ]:
os.makedirs(init_ortho_folder, exist_ok=True)

img_list = sorted(glob(os.path.join(undistorted_folder, '*.tiff')))
cam_list = sorted(glob(os.path.join(ba_folder, '*.tsai')))

for img, cam in tqdm(list(zip(img_list, cam_list))):
    img_out_fn = os.path.join(init_ortho_folder, os.path.splitext(os.path.basename(img))[0] + '_map.tiff')
    cmd = [
        'mapproject',
        '--tr', '0.005',
        '--threads', '12',
        refdem_file, img, cam, img_out_fn
    ]
    subprocess.run(cmd)


## Final stereo

In [ ]:
os.makedirs(final_stereo_folder, exist_ok=True)

img_list = sorted(glob(os.path.join(init_ortho_folder, '*.tiff')))
cam_list = [glob(os.path.join(ba_folder, '*' + os.path.splitext(os.path.basename(x))[0].replace('_map','')) + '*.tsai')[0]
            for x in img_list]

img_pairs = list(zip(img_list[0:-1], img_list[1:]))
cam_pairs = list(zip(cam_list[0:-1], cam_list[1:]))

for i in tqdm(range(0,len(img_pairs))):
    img1, img2 = img_pairs[i]
    cam1, cam2 = cam_pairs[i]

    pair_prefix = os.path.join(
        final_stereo_folder,
        os.path.splitext(os.path.basename(img1))[0] + '__' + os.path.splitext(os.path.basename(img2))[0],
        'run'
        )

    cmd = [
        'parallel_stereo',
        '--threads-singleprocess', '12',
        '--threads-multiprocess', '12',
        img1, img2,
        cam1, cam2,
        pair_prefix,
        refdem_file
    ]
    subprocess.run(cmd)

## Rasterize point clouds

In [ ]:
# Rasterize the point clouds
pc_files = sorted(glob(os.path.join(final_stereo_folder, '*', '*-PC.tif')))
print('Rasterizing the point clouds...')
for pc in tqdm(pc_files):
    cmd = [
        'point2dem',
        '--threads', '12',
        '--tr', '0.01',
        pc
    ]
    subprocess.run(cmd)

## Mosaic DEMs

In [ ]:
dem_fns = sorted(glob(os.path.join(final_stereo_folder, '*', '*DEM.tif')))
print('Creating mosaics of DEM using blended, median, NMAD, and count operators')
for method in tqdm(['median', 'nmad', 'count']):
    dem_fn = os.path.join(final_stereo_folder, f'DEM_mosaic_{method}.tif')
    cmd = [
        'dem_mosaic',
        '--threads', '12',
        '-o', dem_fn
    ] + dem_fns
    if method != 'blended':
        cmd += [f"--{method}"]
    subprocess.run(cmd)


## Align DEM mosaic with reference DEM

In [ ]:
dem_fn = os.path.join(final_stereo_folder, 'DEM_mosaic_median.tif')
os.makedirs(dem_align_folder, exist_ok=True)

cmd = [
    'pc_align',
    '--threads', '12',
    '--max-displacement', '3',
    '--save-transformed-source-points',
    refdem_file, dem_fn,
    '-o', os.path.join(dem_align_folder, 'run')
]
subprocess.run(cmd)

# Rasterize the aligned point cloud
pc_aligned_fn = os.path.join(dem_align_folder, 'run-trans_source.tif')
cmd = [
    'point2dem',
    '--threads', '12',
    '--tr', '0.01',
    pc_aligned_fn
]
subprocess.run(cmd)

## Align cameras

In [ ]:
os.makedirs(cam_align_folder, exist_ok=True)


def transform_pinhole_cameras(camera_file, transform_file, out_folder):
    # Read the camera file
    with open(camera_file, 'r') as f:
        camera_lines = f.read().split('\n')
    # get the camera center and rotation
    camera_C_line = [x for x in camera_lines if 'C =' in x][0]
    camera_C = np.array([float(x) for x in camera_C_line.split(' = ')[1].split(' ')])
    camera_R_line = [x for x in camera_lines if 'R = ' in x][0]
    camera_R = np.array([float(x) for x in camera_R_line.split(' = ')[1].split(' ')]).reshape(3,3)

    # Read the transform
    with open(transform_file, 'r') as f:
        transform_lines = f.read()
    # split by lines and spaces, convert to float, and reshape into 4x4
    transform_matrix = np.array([
        list(map(float, line.split()))
        for line in transform_lines.strip().split('\n')
    ])

    # Extract R and T from transform
    transform_R = transform_matrix[:3, :3]
    transform_T = transform_matrix[:3, 3]

    # Apply the transformation to camera
    camera_C_adj = transform_R @ camera_C + transform_T
    camera_R_adj = transform_R @ camera_R

    # Update the camera
    for i, line in enumerate(camera_lines):
        if 'C = ' in line:
            camera_C_adj_string = ' '.join(list(camera_C_adj.astype(str)))
            camera_lines[i] = f"C = {camera_C_adj_string}"
        elif 'R = ' in line:
            camera_R_adj_string = ' '.join(list(camera_R_adj.ravel().astype(str)))
            camera_lines[i] = f"R = {camera_R_adj_string}"

    
    # Save to file
    camera_lines_string = '\n'.join(camera_lines)
    camera_out_file = os.path.join(out_folder, os.path.basename(camera_file))
    with open(camera_out_file, 'w') as f:
        f.write(camera_lines_string)
    print('Transformed camera saved to file:', camera_out_file)

    return


cam_list = sorted(glob(os.path.join(ba_folder, '*.tsai')))
transform_fn = os.path.join(dem_align_folder, 'run-transform.txt')

for cam in tqdm(cam_list):
    transform_pinhole_cameras(cam, transform_fn, cam_align_folder)


## Mapproject images onto the reference DEM

In [ ]:
os.makedirs(final_ortho_folder, exist_ok=True)

img_list = sorted(glob(os.path.join(undistorted_folder, '*.tiff')))
cam_list = sorted(glob(os.path.join(cam_align_folder, '*.tsai')))

for img, cam in tqdm(list(zip(img_list, cam_list))):
    img_out_fn = os.path.join(final_ortho_folder, os.path.splitext(os.path.basename(img))[0] + '_map.tiff')
    cmd = [
        'mapproject',
        '--tr', '0.005',
        '--threads', '12',
        refdem_file, img, cam, img_out_fn
    ]
    subprocess.run(cmd)

## Mosaic mapprojected images

In [ ]:
img_list = sorted(glob(os.path.join(final_ortho_folder, '*_map.tiff')))
mosaic_fn = os.path.join(final_ortho_folder, 'orthomosaic.tif')
cmd = [
    'image_mosaic',
    '--threads', '12',
    '-o', mosaic_fn
] + img_list
subprocess.run(cmd)

## Plot results

In [ ]:
# Plot the ortho image and DEM mosaics
import matplotlib.pyplot as plt
import xarray as xr

fig_fn = os.path.join(out_folder, 'result.jpg')

# Load the input files
ortho_files = sorted(glob(os.path.join(final_ortho_folder, '*_map.tiff')))

dem_fn = os.path.join(dem_align_folder, 'run-trans_source-DEM.tif')
dem = rxr.open_rasterio(dem_fn).squeeze()
crs = dem.rio.crs
dem = xr.where(dem < -100, np.nan, dem)
dem = dem.rio.write_crs(crs)
refdem = rxr.open_rasterio(refdem_file).squeeze()
refdem = refdem.rio.reproject_match(dem)
refdem = xr.where(refdem < -100, np.nan, refdem)

plt.rcParams.update({'font.sans-serif': 'Verdana', 'font.size': 12})
fig, ax = plt.subplots(1, 3, figsize=(18,8))
# Ortho
for ortho_file in ortho_files:
    ortho = rxr.open_rasterio(ortho_file).squeeze()
    ortho = xr.where(ortho < -1e3, np.nan, ortho)
    ax[0].imshow(
        ortho, 
        cmap='Grays_r',
        extent=(min(ortho.x), max(ortho.x), min(ortho.y), max(ortho.y))
        )
ax[0].set_title('IR image mosaic')
# DEM
im = ax[1].imshow(
    dem, 
    cmap='terrain', 
    extent=(min(dem.x), max(dem.x), min(dem.y), max(dem.y), 'meters')
    )
cb = fig.colorbar(im, shrink=0.5)
ax[1].set_title('DSM mosaic')
# DEM - refdem
im = ax[2].imshow(
    dem - refdem, 
    cmap='coolwarm_r',
    clim=(-1,1),
    extent=(min(dem.x), max(dem.x), min(dem.y), max(dem.y))
    )
cb = fig.colorbar(im, shrink=0.5, label='meters')
ax[2].set_title('Lidar - DSM')

ax[0].set_xlim(ax[1].set_xlim())
ax[0].set_ylim(ax[1].get_ylim())

fig.tight_layout()
plt.show()

fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
# Try 